# SSPINN

## Training and Predicting

** Note that these examples are not expected to run properly in a jupyter notebook*

In [5]:
from sspinn.predict import Predictor as Pred
from sspinn.predict import Net
from sspinn.nn_translator import nn_translator as nnt
from sspinn.visualizer import Visualizer as Vis

### Train

If you want to, you can set up your own neural network to train using Karas. However, if you just run SSPINN as provided, it will use the neural network that we have set up in the class Net, the details of which will be used as a model to describe how to set up a neural network below.

In [ ]:
nn = keras.models.Sequential()

The neural net that we have set up has three layers:

The first layer is a dense layer with 1000 nodes that uses the exponential linear unit (elu) function, where half of the nodes are dropped.

In [ ]:
nn.add(keras.layers.Dense(1000, input_dim=_N_DIMS_IN))
nn.add(keras.layers.Activation('elu'))
nn.add(keras.layers.Dropout(0.5))

The second layer is a dense layer with 3000 nodes that uses a sigmoid function, where 30 percent of the nodes are dropped.

In [ ]:
nn.add(keras.layers.Dense(3000))
nn.add(keras.layers.Activation('sigmoid'))
nn.add(keras.layers.Dropout(0.3))

And the third layer, the output layer, has the same number of nodes as number of elements in the output matrix (186,624 for our 432 by 432 matrix) and uses a rectified linear unit (relu) function.

In [ ]:
nn.add(keras.layers.Dense(_N_DIMS_OUT))
nn.add(keras.layers.Activation('relu'))

Finally, your neural network will need to be compiled using the following function:

In [ ]:
nn.compile(optimizer='sgd', loss='mean_squared_error')

This is what comprises out `Net` class.

Then the `Predictor` class takes the `Net` and trains it using the data files in the data directory provided.

If you are using your own neural network you would need to first create an instance of `Predictor` and the `run set_nn(nn)` to set your custom neural network:

In [ ]:
t = Predictor(train=True, data_dir='./path/to/training/data/dir',
              net='whereTheTrainedNetWill.h5')
t.set_nn(nn)

Now you are ready to actually train the neural network, which can be done using the function `train()`, where any additional keyword arguments will be passed to the Karas train function:

In [ ]:
t.train(filepath='../sspinn/nets/yourNetName', epochs=10000,
        validation_split=0.2, **kwargs)

This will give you a trained neural network, which you can then use to predict the structures of other molecules given C-NMR spectra. 

** We have found that each epoch takes about 10 minutes to run on our CPU and that each epoch takes to much memory to run on a GPU with 8GB of memory. In addition, the size of our trained neural network is upwards of 4GB*

### Predict

Now that you have a trained neural network, you can use `Predictor` to get new predictions for C-NMR spectra. Again, we will first make an instance of `Predictor`:

In [ ]:
p = Predictor(train=False, data_dir='./path/to/testing/data/dir',
              net='whereTheTrainedNetLives.h5')

You need to get correctly formatted input to feed into the neural net, and can do so using `nn_translator(inputFile)` where `nn_input` will be the input to the neural net, `input_matrix` will be `None` since `train=False` and `empirical` will be the dictionary containing the empirical formula of the molecule:

In [ ]:
nn_input, intput_matrix, emperical = nnt(inputfile, train=False)
nn_input = np.array(nn_input).reshape((1, 3349))

Now, you can get the connectivity matrix of the predicted structure using the following command, which is calling the Keras predict method on the neural network that you have created.

In [ ]:
nn_output = p._nn.predict(nn_input)

From here you can use the `Visualizer` class to create an image of the predicted structure:

In [ ]:
v = Vis(nn_output, empirical)
v.draw2Dstructure()